In [11]:
import sys
sys.path.append('../')

In [13]:
from src.data_ import DataPrep
from src.models_ import instantiate_model, adam
from src.train_eval_ import train_model, evaluate_model
from src.utils_ import set_seed, load_json, evaluate_aerofoil_predictions
from src.plot_ import CurveVisualiser

set_seed(42)

In [15]:
dp = DataPrep('../data/csv/14KP_48CLA.csv')
data = dp.get_data()
train_loader = data["train_loader"]
test_loader = data["test_loader"]
X_scaler = data["X_scaler"]
y_scaler = data["y_scaler"]
input_size = data["input_size"]
output_size = data["output_size"]

params = load_json('../final_model/final_params.json')
learning_rate = params['learning_rate']
num_epochs = params['num_epochs']

In [16]:
model = instantiate_model(
    net_name='GRUModel',
    input_size=input_size,
    output_size=output_size,
    params=params
)

optimiser = adam(model, learning_rate)

In [ ]:
train_losses, val_losses = train_model(
    model=model,
    optimizer=optimiser,
    train_loader=train_loader,
    test_loader=test_loader,
    num_epochs=num_epochs,
    loss_func_name='log_cosh',
    save='gru_log.pt'
)

res = evaluate_model(
    model=model,
    test_loader=test_loader,
    y_scaler=y_scaler
)


In [18]:
print(f"MSE: {res['mse']:.4f}")
print(f"RMSE: {res['rmse']:.4f}")
print(f"MAE: {res['mae']:.4f}")
print(f"R2: {res['r2']:.4f}")

MSE: 0.8104
RMSE: 0.6963
MAE: 0.5057
R2: 0.7791


In [ ]:
sorted(data['test_names'])

In [9]:
cv = CurveVisualiser(predictions=res['predictions'], targets=res['targets'])
cv.plot_training_loss(train_losses=train_losses, val_losses=val_losses)
cv.plot_actual_vs_predicted()
cv.plot_comparison()

In [20]:
metrics =evaluate_aerofoil_predictions(
    csv_file='../data/csv/14KP_48CLA.csv',
    model=model,
    X_scaler=X_scaler,
    y_scaler=y_scaler,
    plot_sample=['dae31']
)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warni

ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'headless' is currently running